In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import warnings
warnings.simplefilter("ignore") 
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
from kinoml.datasets.chembl import ChEMBLDatasetProvider
chembl = ChEMBLDatasetProvider.from_source()

In [5]:
chembl

<ChEMBLDatasetProvider with 203380 pIC50Measurement measurements and 162584 systems>

In [6]:
df = chembl.to_dataframe()
df

,Systems,n_components,Measurement,MeasurementType
0,P00533 & Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F...,2,7.387216,pIC50Measurement
1,P35968 & Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F...,2,4.782516,pIC50Measurement
2,P00533 & Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)...,2,6.769551,pIC50Measurement
3,P06239 & Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C...,2,6.853872,pIC50Measurement
4,P06239 & Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C...,2,5.928118,pIC50Measurement
...,...,...,...,...
203375,P42345 & Nc1cc(C(F)F)c(-c2nc(N3CCOCC3)cc(N3CCO...,2,7.376751,pKiMeasurement
203376,P42345 & Nc1cc(C(F)(F)F)c(-c2cc(N3C4CCC3COC4)n...,2,7.522879,pKiMeasurement
203377,P42345 & Nc1cc(C(F)F)c(-c2cc(N3C4CCC3COC4)nc(N...,2,7.920819,pKiMeasurement
203378,P42345 & Nc1cc(C(F)(F)F)c(-c2nc(N3C4CCC3COC4)c...,2,6.361511,pKiMeasurement


In [7]:
print("Measurements:", len(chembl.measurements))
print("Systems:", len(chembl.systems))
print("Proteins:", len(set([s.protein for s in chembl.systems])))
print("Ligands:", len(set([s.ligand.name for s in chembl.systems])))
print("Measurement types:", *df['MeasurementType'].unique())

Measurements: 203380
Systems: 162584
Proteins: 422
Ligands: 103097
Measurement types: pIC50Measurement pKdMeasurement pKiMeasurement


Having this many ligands (compared to PKIS2) makes this dataset take much more memory and longer (~10 mins) to initialize!

In [8]:
from kinoml.features.ligand import SmilesToLigandFeaturizer, MorganFingerprintFeaturizer
from kinoml.features.protein import AminoAcidCompositionFeaturizer
from kinoml.features.core import ScaleFeaturizer, Concatenated, Pipeline

morgan_featurizer = Pipeline([SmilesToLigandFeaturizer(), MorganFingerprintFeaturizer(nbits=1024, radius=2)])
composition_featurizer = Pipeline([AminoAcidCompositionFeaturizer(), ScaleFeaturizer()])
concat_featurizers = Concatenated([morgan_featurizer, composition_featurizer], axis=0)

You can prefeaturize everything before the loop, or delay the featurization until the systems are needed by passing the featurizer to the `to_pytorch` constructor.

In [9]:
# prefeaturize everything
chembl.featurize(concat_featurizers, processes=6)

RDKit WARNING: [16:30:12] Conflicting single bond directions around double bond at index 14.
RDKit WARNING: [16:30:12]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [16:30:12] Conflicting single bond directions around double bond at index 14.
RDKit WARNING: [16:30:12]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [16:36:42] Conflicting single bond directions around double bond at index 17.
RDKit WARNING: [16:36:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [16:43:30] Conflicting single bond directions around double bond at index 17.
RDKit WARNING: [16:43:30]   BondStereo set to STEREONONE and single bond directions set to NONE.


[<ProteinLigandComplex with 2 components (<AminoAcidSequence name=P43405>, <SmilesLigand name=Cn1nc(-c2cnc3[nH]cc(C(=O)NC(C)(C)C)c3n2)c2ccc(Cl)cc21>)>,
 <ProteinLigandComplex with 2 components (<AminoAcidSequence name=P19784>, <SmilesLigand name=CN1CC2(C1)CN(c1cc(C#N)cc(Nc3nc(NC4CC4)c4ncc(C#N)n4n3)c1Cl)C2>)>,
 <ProteinLigandComplex with 2 components (<AminoAcidSequence name=P10721>, <SmilesLigand name=COc1cc2ncnc(N3CCN(C(=O)Nc4ccc(Br)cc4)CC3)c2cc1OC>)>,
 <ProteinLigandComplex with 2 components (<AminoAcidSequence name=P35968>, <SmilesLigand name=COc1cc(C2=C(c3cc[nH]c3)C(=O)NC2=O)cc(OC)c1OC>)>,
 <ProteinLigandComplex with 2 components (<AminoAcidSequence name=Q16288>, <SmilesLigand name=C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl>)>,
 <ProteinLigandComplex with 2 components (<AminoAcidSequence name=Q9P1W9>, <SmilesLigand name=OCCOc1cncc(-c2cc3c(cn2)cnn3-c2cccc(N3CCNCC3)n2)n1>)>,
 <ProteinLigandComplex with 2 components (<AminoAcidSequence name=P04629>, <SmilesLigand name=Cn

In [10]:
from kinoml.datasets.groups import CallableGrouper
grouper = CallableGrouper(lambda measurement: 'invalid' if 'last' not in measurement.system.featurizations else 'valid')
grouper.assign(chembl, overwrite=True)
groups = chembl.split_by_groups()
len(groups.get('valid', [])), len(groups.get('invalid', []))

(203378, 2)

In [13]:
# pass the featurizer to pytorch to featurize on the fly while optimizing
datasets = groups['valid'].to_pytorch()
datasets

Save to disk using `torch.save` native utilities. By calling `list(torch.data.Dataset())` we store a list of 2-tuples with `X, y` tensors (X being the featurized system, y being the measurement). We create one `.pt` file per measurement type.

In [15]:
import torch
for ds in datasets:
    data = list(ds)
    torch.save(data, f"ChEMBL_{ds.observation_model.__qualname__.split('.')[0]}.pt")

If we wanted to work with a subset of the data, this is one way:

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
def hist_plot(values):
    '''
    Plots the histogram of given values.
    '''
    print(f'Min values: {pd.np.min(values):.2f}')
    print(f'Max values: {pd.np.max(values):.2f}')
    fig, ax = plt.subplots()
    ax.set_title("Histogram")
    ax.set_xlabel("pIC50 values in [M]")
    ax.set_ylabel("counts")
    ax.hist(values, bins=50)
    return fig

hist_plot(subdf.Measurement);

In [16]:
observation_models = chembl.observation_models(backend="pytorch")
observation_models

[<function kinoml.core.measurements.pIC50Measurement._observation_model_pytorch(dG_over_KT, substrate_conc=1e-06, michaelis_constant=1, inhibitor_conc=1, **kwargs)>,
 <function kinoml.core.measurements.pKdMeasurement._observation_model_pytorch(dG_over_KT, inhibitor_conc=1, **kwargs)>,
 <function kinoml.core.measurements.pKiMeasurement._observation_model_pytorch(dG_over_KT, inhibitor_conc=1, **kwargs)>]

## Train/test split

Filters and splitters have been reduced to the same concept: groups! You can find lots of groupers in `kinoml.datasets.groups`. A random split, for example:

In [ ]:
from kinoml.datasets.groups import RandomGrouper

grouper = RandomGrouper(ratios={"train": 0.8, "test": 0.2})
grouper.assign(subchembl, overwrite=True)

In [ ]:
groups = subchembl.split_by_groups()
len(groups["train"]), len(groups["test"])

In [ ]:
import torch
from kinoml.ml.torch_models import NeuralNetworkRegression
from tqdm.auto import trange, tqdm

# Use DataLoader for minibatches
datasets = groups["train"].to_pytorch()
loaders = [dataset.as_dataloader(batch_size=5, shuffle=True) for dataset in datasets]

In [ ]:
# precompute input size
input_size = datasets[0].estimate_input_size()
model = NeuralNetworkRegression(input_size=input_size[0])
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
loss_function = torch.nn.MSELoss() # Mean squared error

nb_epoch = 100
loss_timeseries = []
ys = []
range_epochs = trange(nb_epoch, desc="Epochs (+ featurization...)")
for epoch in range_epochs:
    # Single cumulative loss / or loss per loader? look into this!
    cumulative_loss = 0.0
    ys.append([])
    for i, loader in enumerate(loaders):
        for j, (x, y) in enumerate(loader):

            # Clear gradients
            optimizer.zero_grad()

            # Obtain model prediction given model input
            delta_g = model(x)

            # with observation model
            prediction = loader.dataset.observation_model(delta_g)
            
            # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # !!! Make sure prediction and y match shapes !!!
            # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            
            y = y.reshape(prediction.shape)

            ys[-1].append((delta_g, prediction, y))

            # prediction = delta_g
            loss = loss_function(prediction, y)

            # Obtain loss for the predicted output
            # if cumulative loss is global, change this i to 0, or viceversa
            cumulative_loss += loss.item()

            # Gradients w.r.t. parameters
            loss.backward()

            # Optimizer
            optimizer.step()
            if j % 2000 == 0:    # print every 2000 mini-batches
                range_epochs.set_description(f"Epochs (loss={cumulative_loss / 2000:.2e})")
                cumulative_loss = 0.0
            
    loss_timeseries.append(cumulative_loss)
    

In [ ]:
f = plt.figure()
plt.plot(loss_timeseries)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
import numpy as np
from ipywidgets import interact
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def predicted_vs_true(i=100):
    fig, ax = plt.subplots()
    predicted = np.concatenate([y[1].detach().numpy() for y in ys[i]])
    true = np.concatenate([y[2].detach().numpy() for y in ys[i]]).reshape(-1, 1)
    ax.scatter(predicted, true)
    ax.set(xlim=(0, 15), ylim=(0, 15))
    ax.set_xlabel("Predicted y")
    ax.set_ylabel("True y")
    x = np.linspace(0, 15, 10)
    ax.plot(x, x)
    ax.set_aspect('equal', adjustable='box')
    plt.show()

    r2 = r2_score(true, predicted)
    print(f"R2: Goodness of fit measure: {r2:.2f}")
    if all(elem==predicted[0] for elem in predicted):
        print("All outputs are equal: ")
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mse)
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")

interact(predicted_vs_true, i=(0, len(ys)-1));

Let's see how the model predicts the whole dataset!

In [ ]:
model_input = torch.tensor(datasets[0].systems).type(torch.FloatTensor)
true = datasets[0].measurements

delta_g = model(model_input)
prediction = datasets[0].observation_model(delta_g).detach().numpy()


fig, ax = plt.subplots()
ax.scatter(prediction, true)
ax.set(xlim=(0, 15), ylim=(0, 15))
ax.set_xlabel("Predicted y")
ax.set_ylabel("True y")
ax.set_title(f"pIC50 values for a {len(datasets[0])//1000}k ChEMBL subset (train)")
x = np.linspace(0, 15, 10)
ax.plot(x, x)
ax.set_aspect('equal', adjustable='box')
plt.show()

r2 = r2_score(true, prediction)
print(f"R2: Goodness of fit measure: {r2:.2f}")
if all(elem==prediction[0] for elem in prediction):
    print("All outputs are equal: ")
mse = mean_squared_error(true, prediction)
mae = mean_absolute_error(true, prediction)
rmse = np.sqrt(mse)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

### Performance vs the test set

In [ ]:
test_datasets = groups["test"].to_pytorch()

model_input = torch.tensor(test_datasets[0].systems).type(torch.FloatTensor)
true = test_datasets[0].measurements

delta_g = model(model_input)
prediction = test_datasets[0].observation_model(delta_g).detach().numpy()


fig, ax = plt.subplots()
ax.scatter(prediction, true)
ax.set(xlim=(0, 15), ylim=(0, 15))
ax.set_xlabel("Predicted y")
ax.set_ylabel("True y")
ax.set_title(f"pIC50 values for a {len(test_datasets[0])//1000}k ChEMBL subset (test)")
x = np.linspace(0, 15, 10)
ax.plot(x, x)
ax.set_aspect('equal', adjustable='box')
plt.show()

r2 = r2_score(true, prediction)
print(f"R2: Goodness of fit measure: {r2:.2f}")
if all(elem==prediction[0] for elem in prediction):
    print("All outputs are equal: ")
mse = mean_squared_error(true, prediction)
mae = mean_absolute_error(true, prediction)
rmse = np.sqrt(mse)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

## Gradient boosting algorithm (tree-based)
Let us check how a tree-based method worked compared to the neural network.
We use the XGBoost package for the implementation.

### Create XGBoost format input data

In [ ]:
data_xgb = groups["train"].to_xgboost()[0]
data_xgb

### Custom loss function
Here we define a custom loss function for the observation model.
- `labels` are the $pIC_{50}$ values that are the observed measurements. 
- `preds` are the values that are predicted by the model. In this case, the $\Delta g$ values.
We use the typical squared error as a loss:

$$
loss(y, \hat y) = 1/2 * (y - \hat y)^2
$$

In this situation, given an observation model, we define the following custom loss

$$
loss(labels, preds) = 1/2 * (labels-observation\_model(preds))^2.
$$

In the $pIC_{50}$ case, the observation model is

$$
\mathbf{F}_{pIC_{50}}(\Delta g) = -\log10\Big(\big({1+\frac{[S]}{K_m}}\big) * \exp[\Delta g] * C\Big),
$$

or written differently

$$
\mathbf{F}_{pIC_{50}}(\Delta g) = - \frac{\Delta g + \ln\Big(\big(1+\frac{[S]}{K_m}\big)*C\Big)}{\ln(10)}.
$$

Therefore, we have

$$
loss(labels, \Delta g) = 1/2 * \big(labels - \mathbf{F}_{pIC_{50}}(\Delta g) \big) ^2 \\
=  1/2 * \Big(labels +  \frac{\Delta g + \ln\Big(\big(1+\frac{[S]}{K_m}\big)*C\Big)}{\ln(10)} \Big)^2 \\
=  1/2 * \big(labels +  \frac{\Delta g}{\ln(10)} + K \big)^2,
$$
where $K = \frac{\ln\Big(\big(1+\frac{[S]}{K_m}\big)*C\Big)}{\ln(10)}$.

The gradient of the loss w.r.t $\Delta g$ is

$$
\frac{\partial loss}{\Delta g}(labels, \Delta g) = \big(labels +  \frac{\Delta g}{\ln(10)} + K \big) * \frac{1}{\ln(10)}
$$

The hessian of the loss w.r.t $\Delta g$ is

$$
\frac{\partial^2 loss}{\Delta g ^2}(labels, \Delta g) = \frac{1}{\ln(10)^2}
$$

In [ ]:
def custom_loss(preds, dtrain):
    '''
    loss = 1/2 * (observation_pIC50(preds)-labels)^2
    '''
    import numpy as np
    
    substrate_conc=1e-6
    michaelis_constant=1
    inhibitor_conc=1
    
    
    labels = dtrain.get_label()
    
    constant = np.log((1 + substrate_conc / michaelis_constant) * inhibitor_conc) / np.log(10)
    
    grad = (labels + preds/np.log(10) + constant) * 1/np.log(10)
    hess = np.ones(grad.shape) * 1/np.log(10)**2
    
    return grad, hess

We have also included the observation model (plus the MSE loss) in the MeasurementType classes, so we can also do:

In [ ]:
obj_function = groups["train"].observation_model(backend="xgboost")
obj_function

### Define model with the custom loss

In [ ]:
import xgboost as xgb
params = {'learning_rate': 1.0}
model = xgb.train(dtrain=data_xgb, params=params, obj=obj_function)

Evaluate the model:

In [ ]:
deltag_train = model.predict(data_xgb)
prediction = datasets[0].observation_model(deltag_train)

true = data_xgb.get_label()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(prediction, true)
ax.set(xlim=(0, 15), ylim=(0, 15))
ax.set_xlabel("Predicted y")
ax.set_ylabel("True y")
ax.set_title(f"pIC50 values for a {len(datasets[0])//1000}k ChEMBL subset (train)")
x = np.linspace(0, 15, 10)
ax.plot(x, x)
ax.set_aspect('equal', adjustable='box')
plt.show()

r2 = r2_score(true, prediction)
print(f"R2: Goodness of fit measure: {r2:.2f}")
if all(elem==prediction[0] for elem in prediction):
    print("All outputs are equal: ")
mse = mean_squared_error(true, prediction)
mae = mean_absolute_error(true, prediction)
rmse = np.sqrt(mse)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

#### Performance vs test

In [ ]:
data_xgb_test = groups["test"].to_xgboost()[0]
deltag_test = model.predict(data_xgb_test)
prediction_test = datasets[0].observation_model(deltag_test)

true = data_xgb_test.get_label()

fig, ax = plt.subplots()
ax.scatter(prediction_test, true)
ax.set(xlim=(0, 15), ylim=(0, 15))
ax.set_xlabel("Predicted y")
ax.set_ylabel("True y")
ax.set_title(f"pIC50 values for a {len(groups['test'])//1000}k ChEMBL subset (test)")
x = np.linspace(0, 15, 10)
ax.plot(x, x)
ax.set_aspect('equal', adjustable='box')
plt.show()

r2 = r2_score(true, prediction_test)
print(f"R2: Goodness of fit measure: {r2:.2f}")
if all(elem==prediction_test[0] for elem in prediction_test):
    print("All outputs are equal: ")
mse = mean_squared_error(true, prediction_test)
mae = mean_absolute_error(true, prediction_test)
rmse = np.sqrt(mse)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")